In [103]:
import pandas as pd
import numpy as np
import string as st
from afinn import Afinn
# import matplotlib.pylab as plt
# import seaborn as sns
# plt.style.use('ggplot')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows',800)
df_rev=pd.read_csv('../Data/clean_googleplaystore_user_reviews.csv')
df_app=pd.read_csv('../Data/cleaned_googleplaystore.csv')


In [40]:
df_app

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.00,10000,Free,0.0,Everyone,Art & Design
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.70,5000000,Free,0.0,Everyone,Art & Design
2,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.00,50000000,Free,0.0,Teen,Art & Design
3,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.80,100000,Free,0.0,Everyone,Art & Design;Creativity
4,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.60,50000,Free,0.0,Everyone,Art & Design
...,...,...,...,...,...,...,...,...,...,...
8192,FR Calculator,FAMILY,4.0,7,2.60,500,Free,0.0,Everyone,Education
8193,Sya9a Maroc - FR,FAMILY,4.5,38,53.00,5000,Free,0.0,Everyone,Education
8194,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.60,100,Free,0.0,Everyone,Education
8195,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,-99.99,1000,Free,0.0,Mature 17+,Books & Reference


In [ ]:
df_app[['Content Rating','Type','Price']].query("Type=='Paid'").groupby('Content Rating').mean('Price')

In [ ]:
df_rev.info()

TASK_2 - AFINN METHOD (VAIBHAV)

In [104]:
af = Afinn()

# Compute sentiment scores and categories
sentiment_scores = [af.score(str(text)) for text in df_rev.Translated_Review]
sentiment_category = ['positive' if score > 0 else 'negative' if score < 0 else 'neutral' for score in sentiment_scores]
df = pd.DataFrame([sentiment_scores, sentiment_category]).T # .T: swap rows and cols
df.columns = ['AFINN_sentiment_score', 'AFINN_sentiment_category']
df_rev  = pd.concat([df_rev, df], axis=1)
df_rev

,App,Translated_Review,AFINN_sentiment_score,AFINN_sentiment_category
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,13.0,positive
1,10 Best Foods for You,This help eating healthy exercise regular basis,4.0,positive
2,10 Best Foods for You,NOREVIEW,0.0,neutral
3,10 Best Foods for You,Works great especially going grocery store,3.0,positive
4,10 Best Foods for You,Best idea us,3.0,positive
...,...,...,...,...
41705,Housing-Real Estate & Property,NOREVIEW,0.0,neutral
41706,Housing-Real Estate & Property,NOREVIEW,0.0,neutral
41707,Housing-Real Estate & Property,NOREVIEW,0.0,neutral
41708,Housing-Real Estate & Property,NOREVIEW,0.0,neutral


TASK_2 - NP METHOD (ONUR)

In [105]:
df_p=pd.read_excel('../Data/p.xlsx',header=None)
df_n=pd.read_excel('../Data/n.xlsx',header=None)

np_dict=dict.fromkeys(list(df_p[0]), 1)
np_dict.update(dict.fromkeys(list(df_n[0]), -1))
len(np_dict)

6785

In [106]:
df_rev['NP_sentiment_score']=pd.NA
df_rev['NP_sentiment_category']=pd.NA

In [ ]:
df_rev.head()

In [ ]:
# import logging

# logging.basicConfig(filename='log.txt',
#                     filemode='a',
#                     format='%(message)s',
#                     level=logging.ERROR)
def sentiment_category(s_score):
    if s_score<0:
        return 'Negative'
    elif s_score>0:
        return 'Positive'
    else:
        return 'Neutral'
for i in range(len(df_rev['Translated_Review'])):
    sentiment_score=0
    review=df_rev['Translated_Review'][i].translate(str.maketrans('','',st.punctuation)).lower().split()
    for word in review: #range(len(review)):
        try:
            sentiment_score+=np_dict[word]
        except(KeyError):
            # logging.error(f'{review[z]}')
            pass
    df_rev['NP_sentiment_score'][i]=sentiment_score
    df_rev['NP_sentiment_category'][i]=sentiment_category(sentiment_score)

df_rev

In [ ]:
df_rev['NP_sentiment_category'].value_counts()


In [ ]:
df_rev['AFINN_sentiment_category'].value_counts()


TASK_3 -for paid apps only list the top 5 highest and lowest sentiment numbers for afinn column with the name of the app and the app category

In [108]:
df_rev=df_rev.merge(df_app[['App','Category','Type']],on='App')

df_rev[['App','Type']].groupby('App')

In [113]:
df_rev.groupby('App')[['Type']].value_counts()



In [129]:
df_rev[df_rev['App']=='I Am Rich']

,App,Translated_Review,AFINN_sentiment_score,AFINN_sentiment_category,NP_sentiment_score,NP_sentiment_category,Category,Type


In [130]:
df_app[df_app['App']=='I Am Rich']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres
3957,I Am Rich,FAMILY,3.6,217,4.9,10000,Paid,389.99,Everyone,Entertainment


In [128]:
list(df_rev['App'].unique()), sorted(list(df_app.query("Type=='Paid'")['App'].unique()))

(['10 Best Foods for You',
  '11st',
  '1LINE – One Line with One Touch',
  '2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif',
  '2Date Dating App, Love and matching',
  '2RedBeans',
  '30 Day Fitness Challenge - Workout at Home',
  '365Scores - Live Scores',
  '3D Live Neon Weed Launcher',
  '4 in a Row',
  '4K Wallpapers and Ultra HD Backgrounds',
  '7 Cups: Anxiety & Stress Chat',
  '7 Day Food Journal Challenge',
  '7 Minute Workout',
  '7 Weeks - Habit & Goal Tracker',
  '8 Ball Pool',
  '850 Sports News Digest',
  '8fit Workouts & Meal Planner',
  '95Live -SG#1 Live Streaming App',
  'A Call From Santa Claus!',
  'A Word A Day',
  'A&E - Watch Full Episodes of TV Shows',
  'A+ Gallery - Photos & Videos',
  'A+ Mobile',
  'ABC Kids - Tracing & Phonics',
  'ABC News - US & World News',
  'ABC Preschool Free',
  'ABCmouse.com',
  'AC - Tips & News for Android™',
  'ACE Elite',
  'AD - Nieuws, Sport, Regio & Entertainment',
  'AMC Theatres',
  'AOL - News, Mail & Video',
  'AP Mobil

In [90]:
df_rev.query("Type=='Paid'")['App'].value_counts()

Dr. Panda & Toto's Treehouse      44
Diabetes & Diet Tracker           43
Beautiful Widgets Pro             40
Call of Duty:Black Ops Zombies    40
Farming Simulator 18              40
HD Widgets                        40
Name: App, dtype: int64

In [85]:
df_rev.head(100)

,App,Translated_Review,AFINN_sentiment_score,AFINN_sentiment_category,NP_sentiment_score,NP_sentiment_category,Category,Type
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,13.0,positive,4,Positive,HEALTH_AND_FITNESS,Free
1,10 Best Foods for You,This help eating healthy exercise regular basis,4.0,positive,1,Positive,HEALTH_AND_FITNESS,Free
2,10 Best Foods for You,NOREVIEW,0.0,neutral,0,Neutral,HEALTH_AND_FITNESS,Free
3,10 Best Foods for You,Works great especially going grocery store,3.0,positive,2,Positive,HEALTH_AND_FITNESS,Free
4,10 Best Foods for You,Best idea us,3.0,positive,1,Positive,HEALTH_AND_FITNESS,Free
5,10 Best Foods for You,Best way,3.0,positive,1,Positive,HEALTH_AND_FITNESS,Free
6,10 Best Foods for You,Amazing,4.0,positive,1,Positive,HEALTH_AND_FITNESS,Free
7,10 Best Foods for You,NOREVIEW,0.0,neutral,0,Neutral,HEALTH_AND_FITNESS,Free
8,10 Best Foods for You,"Looking forward app,",0.0,neutral,0,Neutral,HEALTH_AND_FITNESS,Free
9,10 Best Foods for You,It helpful site ! It help foods get !,4.0,positive,1,Positive,HEALTH_AND_FITNESS,Free


In [131]:
df_rev.query("Type=='Paid'").groupby(['App'])[['App','AFINN_sentiment_score']].value_counts()


App                             AFINN_sentiment_score
Beautiful Widgets Pro            0.0                      8
                                 2.0                      4
                                -1.0                      4
                                 1.0                      3
                                 3.0                      3
                                -2.0                      3
                                 6.0                      3
                                -6.0                      2
                                 4.0                      2
                                 5.0                      2
                                 10.0                     2
                                -3.0                      1
                                 20.0                     1
                                 11.0                     1
                                 9.0                      1
Call of Duty:Black Ops Zombies   0.0          

TASK_4 - what is the best category according to afinn sentiment values